In [0]:
from confluent_kafka import Consumer
from time import sleep
import uuid
from confluent_kafka import Producer, Consumer, KafkaError, KafkaException
from confluent_kafka.admin import AdminClient, NewTopic
import json
import pandas as pd

In [0]:
%run "../includes/Touristconfiguration"

In [0]:
# CREATE TABLE TouristAttractions (
#     Address varchar(100) null,
#     Attraction_name varchar(100) not null,
#     Website varchar(100) null,
#     attraction_lat decimal(9,7) null,
#     attraction_lng decimal(9,7) null,
#     Zipcode varchar(10) null,
#     Neighborhood vachar(15) null
#     msg_timestamp bigint not null
# )

In [0]:
from confluent_kafka import Consumer
from time import sleep
import uuid
from confluent_kafka import Producer, Consumer, KafkaError, KafkaException, TopicPartition
import json

#Kakfa Class Setup.
c = Consumer({
    'bootstrap.servers': confluentBootstrapServers,
    'sasl.mechanism': 'PLAIN',
    'security.protocol': 'SASL_SSL',
    'sasl.username': confluentApiKey,
    'sasl.password': confluentSecret,
    'group.id': topic_group_id,
    'auto.offset.reset': 'earliest',
    #"enable.auto.commit": "false", #added this
    'error_cb': error_cb,
    'batch.num.messages': 1000
})

topic_part = TopicPartition(confluentTopicName, 0)
c.assign([topic_part])

c.subscribe([confluentTopicName])

In [0]:
aJson = {}

kafkaListDictionaries = []

while(True):
    try:
        msg = c.poll(timeout=5.0)
        if msg is None:
            break
        elif msg.error():
            print("Consumer error: {}".format(msg.error()))
            break
        else:
            aJson = json.loads('{}'.format(msg.value().decode('utf-8')))
            aJson['msg_timestamp'] = msg.timestamp()[1]
            display(aJson)
            kafkaListDictionaries.append(aJson)
    except Exception as e:
        print(e)

{'Address': '14 E 60th St, Central Park Conservancy, New York City, NY 10022',
 'Attraction': 'Central Park',
 'Website': 'https://www.centralparknyc.org',
 'latitude': 40.764321,
 'longitude': -73.97182,
 'Zipcode': 10022.0,
 'Neighborhood': 'Manhattan',
 'msg_timestamp': 1684225255031}{'Address': '20 W 34th St, New York City, NY 10001',
 'Attraction': 'Empire State Building',
 'Website': 'https://www.esbnyc.com',
 'latitude': 40.748745,
 'longitude': -73.985709,
 'Zipcode': 10001.0,
 'Neighborhood': 'Manhattan',
 'msg_timestamp': 1684225260036}{'Address': 'Liberty Island, New York City, NY 10004',
 'Attraction': 'Statue of Liberty',
 'Website': 'https://www.nps.gov/stli/index.htm',
 'latitude': 40.689555,
 'longitude': -74.044949,
 'Zipcode': 10004.0,
 'Neighborhood': 'Manhattan',
 'msg_timestamp': 1684225265041}{'Address': 'Manhattan, NY 10036',
 'Attraction': 'Times Square',
 'Website': None,
 'latitude': 40.760249,
 'longitude': -73.99329,
 'Zipcode': 10036.0,
 'Neighborhood': 'Ma

In [0]:
if len(kafkaListDictionaries) == 0: dbutils.notebook.exit("There are no messages to process")

In [0]:
consumer_df = spark.createDataFrame(kafkaListDictionaries)

consumer_df = consumer_df.replace(float('nan'), None)

# display(consumer_df)

In [0]:

table = db_table
user = db_login
password = db_password
server = server
database = database



consumer_df.write.format("jdbc").option(
    "url", f"jdbc:sqlserver://{server}:1433;databaseName={database};"
    ) \
    .mode("append") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()